> Submission for https://www.kaggle.com/competitions/spaceship-titanic/overview

In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [42]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [43]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [44]:
train_df.groupby('Transported').Transported.count() / train_df.Transported.count()

Transported
False    0.496376
True     0.503624
Name: Transported, dtype: float64

### Filling NaN values

In [45]:
train_df.count()

PassengerId     8693
HomePlanet      8492
CryoSleep       8476
Cabin           8494
Destination     8511
Age             8514
VIP             8490
RoomService     8512
FoodCourt       8510
ShoppingMall    8485
Spa             8510
VRDeck          8505
Name            8493
Transported     8693
dtype: int64

In [46]:
train_df = train_df[~train_df.PassengerId.isnull()]
train_df = train_df[~train_df.VIP.isnull()]
train_df = train_df[~train_df.CryoSleep.isnull()]

In [47]:
train_df = train_df.dropna(subset=['Cabin'])
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [48]:
def get_na_null_counts(funct_df):
    return pd.DataFrame(data={
        'isNa': funct_df.isna().sum(),
        'total': funct_df.count(),
        'portion_missing': funct_df.isna().sum()/funct_df.count(),
        'type': funct_df.dtypes
    })

def get_null_counts(funct_df):
    return pd.DataFrame(data={
        'isNa': funct_df.isnull().sum(),
        'total': funct_df.count(),
        'portion_missing': funct_df.isnull().sum()/funct_df.count(),
        'type': funct_df.dtypes
    })

get_na_null_counts(train_df)

,isNa,total,portion_missing,type
PassengerId,0,8094,0.000000,object
HomePlanet,190,7904,0.024038,object
CryoSleep,0,8094,0.000000,object
Cabin,0,8094,0.000000,object
Destination,171,7923,0.021583,object
Age,169,7925,0.021325,float64
VIP,0,8094,0.000000,object
RoomService,171,7923,0.021583,float64
FoodCourt,171,7923,0.021583,float64
ShoppingMall,195,7899,0.024687,float64


In [49]:
train_df_nonna = train_df.copy()
train_df_nonna['HomePlanet'] = train_df.HomePlanet.fillna(train_df.HomePlanet.mode()[0])
train_df_nonna['CryoSleep'] = train_df.CryoSleep.fillna(train_df.CryoSleep.mode()[0])
train_df_nonna['Destination'] = train_df.Destination.fillna(train_df.Destination.mode()[0])
train_df_nonna['VIP'] = train_df.VIP.fillna(train_df.VIP.mode()[0])

train_df_nonna['Age'] = train_df.Age.fillna(train_df.Age.median())
train_df_nonna['RoomService'] = train_df.RoomService.fillna(train_df.RoomService.median())
train_df_nonna['FoodCourt'] = train_df.FoodCourt.fillna(train_df.FoodCourt.median())
train_df_nonna['ShoppingMall'] = train_df.ShoppingMall.fillna(train_df.ShoppingMall.median())
train_df_nonna['Spa'] = train_df.Spa.fillna(train_df.Spa.median())
train_df_nonna['VRDeck'] = train_df.VRDeck.fillna(train_df.VRDeck.median())

train_df_nonna['CryoSleep'] = train_df['CryoSleep'] * 1
train_df_nonna['VIP'] = train_df.VIP * 1
train_df_nonna['Transported'] = train_df.Transported.astype(int)

train_df_nonna = train_df_nonna.drop('Name', axis=1)

train_df_nonna

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0,A/98/P,55 Cancri e,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0
8689,9278_01,Earth,1,G/1499/S,PSO J318.5-22,18.0,0,0.0,0.0,0.0,0.0,0.0,0
8690,9279_01,Earth,0,G/1500/S,TRAPPIST-1e,26.0,0,0.0,0.0,1872.0,1.0,0.0,1
8691,9280_01,Europa,0,E/608/S,55 Cancri e,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0


In [50]:
get_na_null_counts(train_df_nonna)

,isNa,total,portion_missing,type
PassengerId,0,8094,0.0,object
HomePlanet,0,8094,0.0,object
CryoSleep,0,8094,0.0,object
Cabin,0,8094,0.0,object
Destination,0,8094,0.0,object
Age,0,8094,0.0,float64
VIP,0,8094,0.0,object
RoomService,0,8094,0.0,float64
FoodCourt,0,8094,0.0,float64
ShoppingMall,0,8094,0.0,float64


### Check if Passenger group has anything to do with Transported

In [51]:
def get_group_and_pid(row):
    split_out = row.PassengerId.split('_')
    row['group'] = split_out[0]
    row['pp'] = split_out[1]
    
#     print(row.Cabin)
    split_out = row.Cabin.split('/')
#     print(split_out)
    row['cabin_deck'] = split_out[0]
    row['cabin_num'] = split_out[1]
    row['cabin_side'] = split_out[2]
    
    return row
    
new_train_df = train_df_nonna.apply(get_group_and_pid, axis=1)
new_train_df = new_train_df.drop('Cabin', axis=1)
new_train_df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,group,pp,cabin_deck,cabin_num,cabin_side
0,0001_01,Europa,0,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,0001,01,B,0,P
1,0002_01,Earth,0,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,0002,01,F,0,S
2,0003_01,Europa,0,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,0003,01,A,0,S
3,0003_02,Europa,0,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,0003,02,A,0,S
4,0004_01,Earth,0,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,0004,01,F,1,S


In [52]:
get_na_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8094,0.0,object
HomePlanet,0,8094,0.0,object
CryoSleep,0,8094,0.0,int64
Destination,0,8094,0.0,object
Age,0,8094,0.0,float64
VIP,0,8094,0.0,int64
RoomService,0,8094,0.0,float64
FoodCourt,0,8094,0.0,float64
ShoppingMall,0,8094,0.0,float64
Spa,0,8094,0.0,float64


In [53]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8094,0.0,object
HomePlanet,0,8094,0.0,object
CryoSleep,0,8094,0.0,int64
Destination,0,8094,0.0,object
Age,0,8094,0.0,float64
VIP,0,8094,0.0,int64
RoomService,0,8094,0.0,float64
FoodCourt,0,8094,0.0,float64
ShoppingMall,0,8094,0.0,float64
Spa,0,8094,0.0,float64


In [54]:
new_train_df[new_train_df.VIP.isnull()]

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,group,pp,cabin_deck,cabin_num,cabin_side


In [55]:
corr = new_train_df[[
    'group', 'Transported'
]].corr()
corr

,group,Transported
group,1.000000,0.022252
Transported,0.022252,1.000000


In [56]:
backup = new_train_df.copy()

In [57]:
new_train_df = backup.copy()

> Therefore we can conclude that group has no relations with Transported.

### Convert Categorical variables to OneHot

In [58]:
def convert_cat_to_onehot(col, new_train_df):
#     print(new_train_df)
    enc = OneHotEncoder().fit(new_train_df[[col]])
    transformed = enc.transform(new_train_df[[col]]).toarray()
    transformed_df = pd.DataFrame(
        transformed,
        columns=[f"{col}_{i}" for i in enc.categories_[0]]
    )
    new_train_df.dropna(subset=[col])
    new_train_df = pd.concat([new_train_df, transformed_df], axis=1)
    new_train_df = new_train_df.drop(col, axis=1)
    return new_train_df

In [59]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8094,0.0,object
HomePlanet,0,8094,0.0,object
CryoSleep,0,8094,0.0,int64
Destination,0,8094,0.0,object
Age,0,8094,0.0,float64
VIP,0,8094,0.0,int64
RoomService,0,8094,0.0,float64
FoodCourt,0,8094,0.0,float64
ShoppingMall,0,8094,0.0,float64
Spa,0,8094,0.0,float64


In [60]:
print(new_train_df.HomePlanet.unique())
print(new_train_df.Destination.unique())
print(new_train_df.cabin_deck.unique())
print(new_train_df.cabin_side.unique())

['Europa' 'Earth' 'Mars']
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e']
['B' 'F' 'A' 'G' 'E' 'D' 'C' 'T']
['P' 'S']


In [61]:
# enc = OneHotEncoder().fit(new_train_df[['cabin_deck']].values)
# enc.categories_

In [62]:
new_train_df = convert_cat_to_onehot('HomePlanet', new_train_df)
new_train_df = convert_cat_to_onehot('Destination', new_train_df)
new_train_df = convert_cat_to_onehot('cabin_deck', new_train_df)
new_train_df = convert_cat_to_onehot('cabin_side', new_train_df)

In [63]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,599,8094,0.074005,object
CryoSleep,599,8094,0.074005,float64
Age,599,8094,0.074005,float64
VIP,599,8094,0.074005,float64
RoomService,599,8094,0.074005,float64
FoodCourt,599,8094,0.074005,float64
ShoppingMall,599,8094,0.074005,float64
Spa,599,8094,0.074005,float64
VRDeck,599,8094,0.074005,float64
Transported,599,8094,0.074005,float64


In [64]:
# new_train_df.dropna(['PassengerId'], axis=0)
new_train_df = new_train_df.dropna(subset=['PassengerId','HomePlanet_Earth'])
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,7535,0.0,object
CryoSleep,0,7535,0.0,float64
Age,0,7535,0.0,float64
VIP,0,7535,0.0,float64
RoomService,0,7535,0.0,float64
FoodCourt,0,7535,0.0,float64
ShoppingMall,0,7535,0.0,float64
Spa,0,7535,0.0,float64
VRDeck,0,7535,0.0,float64
Transported,0,7535,0.0,float64


In [65]:
new_train_df.columns

Index(['PassengerId', 'CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'group', 'pp',
       'cabin_num', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'Destination_nan', 'cabin_deck_A',
       'cabin_deck_B', 'cabin_deck_C', 'cabin_deck_D', 'cabin_deck_E',
       'cabin_deck_F', 'cabin_deck_G', 'cabin_deck_T', 'cabin_deck_nan',
       'cabin_side_P', 'cabin_side_S', 'cabin_side_nan'],
      dtype='object')

In [66]:
new_train_df.Destination_nan.unique()

array([0.])

In [67]:
new_train_df = new_train_df.drop(
    ['cabin_side_nan', 'Destination_nan', 'cabin_deck_nan'],
    axis=1
)

### Correlations

In [68]:
new_train_df.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,cabin_deck_A,cabin_deck_B,cabin_deck_C,cabin_deck_D,cabin_deck_E,cabin_deck_F,cabin_deck_G,cabin_deck_T,cabin_side_P,cabin_side_S
0,0001_01,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0002_01,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0003_01,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0003_02,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0004_01,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [69]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,7535,0.0,object
CryoSleep,0,7535,0.0,float64
Age,0,7535,0.0,float64
VIP,0,7535,0.0,float64
RoomService,0,7535,0.0,float64
FoodCourt,0,7535,0.0,float64
ShoppingMall,0,7535,0.0,float64
Spa,0,7535,0.0,float64
VRDeck,0,7535,0.0,float64
Transported,0,7535,0.0,float64


In [70]:
get_na_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,7535,0.0,object
CryoSleep,0,7535,0.0,float64
Age,0,7535,0.0,float64
VIP,0,7535,0.0,float64
RoomService,0,7535,0.0,float64
FoodCourt,0,7535,0.0,float64
ShoppingMall,0,7535,0.0,float64
Spa,0,7535,0.0,float64
VRDeck,0,7535,0.0,float64
Transported,0,7535,0.0,float64


In [71]:
corr = new_train_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,group,pp,cabin_num,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,cabin_deck_A,cabin_deck_B,cabin_deck_C,cabin_deck_D,cabin_deck_E,cabin_deck_F,cabin_deck_G,cabin_deck_T,cabin_side_P,cabin_side_S
PassengerId,1.000000,-0.000496,-0.001327,0.013315,-0.002583,-0.008956,0.017868,0.001537,0.018214,0.035580,1.000000,-0.011670,0.674991,0.002496,-0.002652,-0.000261,-0.004448,0.012061,-0.003662,0.005456,-0.014859,0.004778,-0.005203,-0.002841,0.004756,0.004749,-0.017374,0.013146,-0.013146
CryoSleep,-0.000496,1.000000,-0.070610,-0.080315,-0.257381,-0.207173,-0.228009,-0.200772,-0.197465,0.468468,-0.000496,0.071680,-0.034609,0.007680,-0.002570,-0.006753,-0.003279,0.001162,0.002171,-0.016723,0.001851,0.004620,-0.019177,0.013569,0.008210,-0.006665,0.026800,0.002938,-0.002938
Age,-0.001327,-0.070610,1.000000,0.095935,0.071843,0.132623,0.036764,0.121584,0.101114,-0.074282,-0.001327,-0.131024,-0.124809,0.010975,-0.009428,-0.003523,-0.006723,-0.003853,0.008380,-0.017932,-0.004830,-0.003440,0.008595,0.002524,-0.000534,0.005908,0.011762,0.011922,-0.011922
VIP,0.013315,-0.080315,0.095935,1.000000,0.041242,0.123794,0.020732,0.055601,0.119249,-0.033544,0.013315,-0.000153,-0.093533,-0.008319,-0.001144,0.011496,-0.007358,0.014958,-0.002911,0.003081,0.002910,-0.000861,-0.007662,0.016292,0.012552,-0.022227,-0.003086,0.016051,-0.016051
RoomService,-0.002583,-0.257381,0.071843,0.041242,1.000000,-0.013454,0.065888,0.005347,-0.023846,-0.246034,-0.002583,-0.025531,-0.000098,-0.012646,-0.000327,0.015971,-0.010606,0.010334,0.002878,0.002753,0.006339,-0.003716,-0.009809,-0.001645,0.002247,0.001316,-0.006898,-0.007686,0.007686
FoodCourt,-0.008956,-0.207173,0.132623,0.123794,-0.013454,1.000000,-0.012396,0.214930,0.210052,0.050860,-0.008956,0.022350,-0.183194,0.003692,-0.001231,-0.003251,0.006955,0.007937,-0.011158,-0.000949,-0.004413,0.000936,0.003804,-0.011022,-0.001525,0.009798,-0.005552,0.001162,-0.001162
ShoppingMall,0.017868,-0.228009,0.036764,0.020732,0.065888,-0.012396,1.000000,0.020648,-0.006209,0.009559,0.017868,-0.039420,0.008953,0.003880,0.009007,-0.014379,0.018059,0.020941,-0.029181,-0.001559,0.004890,0.012952,-0.011342,0.009138,-0.003118,-0.007340,-0.006111,0.003787,-0.003787
Spa,0.001537,-0.200772,0.121584,0.055601,0.005347,0.214930,0.020648,1.000000,0.141980,-0.217063,0.001536,0.017617,-0.127376,-0.012031,0.011952,0.002141,0.018140,-0.012836,-0.007971,0.044153,-0.010306,0.010721,-0.017144,-0.003899,-0.008093,0.003095,-0.005381,0.002477,-0.002477
VRDeck,0.018214,-0.197465,0.101114,0.119249,-0.023846,0.210052,-0.006209,0.141980,1.000000,-0.206851,0.018214,0.012944,-0.139182,0.006356,0.006829,-0.015120,0.011974,0.012475,-0.018460,0.005948,0.008532,-0.000181,-0.007250,-0.002081,-0.022758,0.021194,-0.005292,0.000064,-0.000064
Transported,0.035580,0.468468,-0.074282,-0.033544,-0.246034,0.050860,0.009559,-0.217063,-0.206851,1.000000,0.035580,0.076256,-0.037205,0.015857,-0.019990,0.001688,-0.012201,-0.009902,0.017040,-0.016824,-0.001604,-0.003805,0.000977,-0.010726,0.009647,0.006153,0.006407,-0.000770,0.000770


### Multicollinear Checks

In [72]:
# X.values.shape, X.columns, len(X.columns)

In [73]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = new_train_df.copy()
X = X.drop('Transported', axis=1)
X1 = np.array(X.values, dtype=float)
vif = [
    variance_inflation_factor(X1, i) for i in range(len(X.columns))
]

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [74]:
vif = pd.DataFrame()
vif["features"] = X.columns
vif["vif_Factor"] = [variance_inflation_factor(X1, i) for i in range(X.shape[1])]
vif

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_Factor
0,PassengerId,inf
1,CryoSleep,1.296869
2,Age,1.083094
3,VIP,1.044908
4,RoomService,1.091803
5,FoodCourt,1.177224
6,ShoppingMall,1.066852
7,Spa,1.116302
8,VRDeck,1.132273
9,group,inf


In [75]:
def drop_col_and_get_vif(X, col_name):
#     X = X.drop('cabin_deck_A', axis=1)
    X.drop(col_name, axis=1, inplace=True)
    X1 = np.array(X.values, dtype=float)
    vif = [
        variance_inflation_factor(X1, i) for i in range(len(X.columns))
    ]

    vif = pd.DataFrame()
    vif["features"] = X.columns
    vif["vif_Factor"] = [variance_inflation_factor(X1, i) for i in range(X.shape[1])]
    return vif

drop_col_and_get_vif(X, 'cabin_deck_A')

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_Factor
0,PassengerId,inf
1,CryoSleep,1.296869
2,Age,1.083094
3,VIP,1.044908
4,RoomService,1.091803
5,FoodCourt,1.177224
6,ShoppingMall,1.066852
7,Spa,1.116302
8,VRDeck,1.132273
9,group,inf


In [76]:
drop_col_and_get_vif(X, 'HomePlanet_Earth')

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_Factor
0,PassengerId,inf
1,CryoSleep,1.296869
2,Age,1.083094
3,VIP,1.044908
4,RoomService,1.091803
5,FoodCourt,1.177224
6,ShoppingMall,1.066852
7,Spa,1.116302
8,VRDeck,1.132273
9,group,inf


In [77]:
drop_col_and_get_vif(X, 'PassengerId')

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_Factor
0,CryoSleep,1.296869
1,Age,1.083094
2,VIP,1.044908
3,RoomService,1.091803
4,FoodCourt,1.177224
5,ShoppingMall,1.066852
6,Spa,1.116302
7,VRDeck,1.132273
8,group,1.990430
9,pp,1.037399


In [78]:
drop_col_and_get_vif(X, 'cabin_side_P')

,features,vif_Factor
0,CryoSleep,1.296869
1,Age,1.083094
2,VIP,1.044908
3,RoomService,1.091803
4,FoodCourt,1.177224
5,ShoppingMall,1.066852
6,Spa,1.116302
7,VRDeck,1.132273
8,group,1.990430
9,pp,1.037399


In [79]:
drop_col_and_get_vif(X, 'cabin_deck_F')

,features,vif_Factor
0,CryoSleep,1.296674
1,Age,1.082802
2,VIP,1.044877
3,RoomService,1.091801
4,FoodCourt,1.176989
5,ShoppingMall,1.066846
6,Spa,1.115256
7,VRDeck,1.132272
8,group,1.990388
9,pp,1.037388


In [81]:
drop_col_and_get_vif(X, 'Destination_TRAPPIST-1e')

,features,vif_Factor
0,CryoSleep,1.881822
1,Age,3.897992
2,VIP,1.069787
3,RoomService,1.207384
4,FoodCourt,1.262940
5,ShoppingMall,1.150715
6,Spa,1.193832
7,VRDeck,1.207698
8,group,7.780493
9,pp,2.673472


In [82]:
selected_columns = X.columns
selected_columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'group', 'pp', 'cabin_num', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'cabin_deck_B', 'cabin_deck_C',
       'cabin_deck_D', 'cabin_deck_E', 'cabin_deck_G', 'cabin_deck_T',
       'cabin_side_S'],
      dtype='object')

# Logistic Regression

In [83]:
from sklearn.linear_model import LogisticRegression

input_df = new_train_df[selected_columns]
output_df = new_train_df['Transported']

clf = LogisticRegression(random_state=0).fit(input_df, output_df)
clf

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [96]:
# input_df[:2]
clf.predict(input_df[:5]), output_df[:5]

(array([1., 0., 0., 0., 0.]),
 0    0.0
 1    1.0
 2    0.0
 3    0.0
 4    1.0
 Name: Transported, dtype: float64)

In [95]:
clf.score(input_df, output_df)

0.7804910418049105